In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# utility関数


In [2]:
def save_metrics(metrics, epoch=None):
    plt.figure(figsize=(10,8))
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join("metrics", "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join("metrics", "g_loss" + str(epoch) + ".png"))
    plt.close()

In [3]:
# noise[[examples, 100]]から生成した画像をplot_dim(例えば4x4)で表示
def save_imgs(images, plot_dim=(5,12), size=(12,5), epoch=None):
    examples = plot_dim[0]*plot_dim[1]

    # 表示
    fig = plt.figure(figsize=size)
    for i in range(examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((96, 96, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join("generated_figures", str(epoch) + ".png"))
    plt.close()


# モデル

In [4]:
class Generator:
    def __init__(self, depths=[1024,512,256,128], s_size=6):
        self.depths = depths + [3]
        self.s_size = s_size
        self.reuse = False

    def __call__(self, inputs, training=False):
        inputs = tf.convert_to_tensor(inputs)
        with tf.variable_scope('g', reuse=self.reuse):
            # reshape from inputs
            with tf.variable_scope('reshape'):
                outputs = tf.layers.dense(inputs, self.depths[0] * self.s_size * self.s_size)
                outputs = tf.reshape(outputs, [-1, self.s_size, self.s_size, self.depths[0]])
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            # deconvolution (transpose of convolution) x 4
            with tf.variable_scope('deconv1'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[1], [5, 5], strides=(2, 2), padding='SAME')
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv2'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[2], [5, 5], strides=(2, 2), padding='SAME')
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv3'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[3], [5, 5], strides=(2, 2), padding='SAME')
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv4'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[4], [5, 5], strides=(2, 2), padding='SAME')
            # output images
            with tf.variable_scope('tanh'):
                outputs = tf.tanh(outputs, name='outputs')
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [5]:
class Discriminator:
    def __init__(self, depths=[64,128,256,512]):
        self.depths = [3] + depths
        self.reuse = False

    def __call__(self, inputs, training=False, name=''):
        def leaky_relu(x, leak=0.2, name=''):
            return tf.maximum(x, x * leak, name=name)
        outputs = tf.convert_to_tensor(inputs)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            # convolution x 4
            with tf.variable_scope('conv1'):
                outputs = tf.layers.conv2d(outputs, self.depths[1], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv2'):
                outputs = tf.layers.conv2d(outputs, self.depths[2], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv3'):
                outputs = tf.layers.conv2d(outputs, self.depths[3], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv4'):
                outputs = tf.layers.conv2d(outputs, self.depths[4], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('classify'):
                batch_size = outputs.get_shape()[0].value
                reshape = tf.reshape(outputs, [batch_size, -1])
                outputs = tf.layers.dense(reshape, 2, name='outputs')
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return outputs


In [6]:
class DCGAN:
    def __init__(self,
                 batch_size=64, s_size=6, z_dim=100,
                 g_depths=[1024, 512, 256, 128],
                 d_depths=[64, 128, 256, 512]):
        self.batch_size = batch_size
        self.s_size = s_size
        self.z_dim = z_dim
        self.g = Generator(depths=g_depths, s_size=self.s_size)
        self.d = Discriminator(depths=d_depths)
        self.z = tf.random_uniform([self.batch_size, self.z_dim], minval=-1.0, maxval=1.0)

    def loss(self, traindata):
        """build models, calculate losses.
        Args:
            traindata: 4-D Tensor of shape `[batch, height, width, channels]`.
        Returns:
            dict of each models' losses.
        """
        generated = self.g(self.z, training=True)
        g_outputs = self.d(generated, training=True, name='g')
        t_outputs = self.d(traindata, training=True, name='t')
        # add each losses to collection
        tf.add_to_collection(
            'g_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.ones([self.batch_size], dtype=tf.int64),
                    logits=g_outputs)))
        tf.add_to_collection(
            'd_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.ones([self.batch_size], dtype=tf.int64),
                    logits=t_outputs)))
        tf.add_to_collection(
            'd_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.zeros([self.batch_size], dtype=tf.int64),
                    logits=g_outputs)))
        return {
            self.g: tf.add_n(tf.get_collection('g_losses'), name='total_g_loss'),
            self.d: tf.add_n(tf.get_collection('d_losses'), name='total_d_loss'),
        }

    def train(self, losses, learning_rate=0.0002, beta1=0.5):
        """
        Args:
            losses dict.
        Returns:
            train op.
        """
        g_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
        d_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
        g_opt_op = g_opt.minimize(losses[self.g], var_list=self.g.variables)
        d_opt_op = d_opt.minimize(losses[self.d], var_list=self.d.variables)
        with tf.control_dependencies([g_opt_op, d_opt_op]):
            return tf.no_op(name='train')



    def train(self, losses, learning_rate=0.0002, beta1=0.5):
        g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
        d_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)

        g_optimizer = g_optimizer.minimize(losses[self.g], var_list=self.g.variables)
        d_optimizer = d_optimizer.minimize(losses[self.d], var_list=self.d.variables)
        with tf.control_dependencies([g_optimizer, d_optimizer]):
            return tf.no_op(name="train")

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        #save_imgs(images, epoch=epoch)
        return images

In [7]:
import numpy as np

X_train = np.load("irasutoya_10575x96x96x3_jpg.npy")
X_train = X_train/255

z_dim = 100
batch_size = 64
epochs = 200000
display_epoch = 1000
param_save_epoch = 100000
loss = {"dis_loss":[], "gen_loss":[]}

In [8]:
p_noise = tf.placeholder(tf.float32, [None, z_dim])
noise_check = np.random.uniform(-1, 1, size=[60, z_dim]).astype(np.float32)

dcgan = DCGAN()
p_train_images = tf.placeholder(tf.float32, [batch_size, 96, 96, 3])
losses = dcgan.loss(traindata=p_train_images)
train_op = dcgan.train(losses)

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        
        #self.z = tf.random_uniform([self.batch_size, self.z_dim], minval=-1.0, maxval=1.0) # 値域これでいいんだっけ？
        
        # ノイズから生成データを生成
        noise = np.random.uniform(-1, 1, size=[batch_size, z_dim])

        if epoch % display_epoch == 0:
            imgs_gen = dcgan.sample_images(inputs=noise_check).eval()
            print("saving images")
            save_imgs(imgs_gen, epoch=epoch)

        # 訓練データを抜粋
        rand_index = np.random.randint(0, X_train.shape[0], size=batch_size)
        exImgs = X_train[rand_index, :].astype(np.float32)

        _, g_loss_value, d_loss_value = sess.run([train_op, losses[dcgan.g], losses[dcgan.d]], feed_dict={p_train_images: exImgs})


        # 結果をappend
        loss["dis_loss"].append(d_loss_value)
        loss["gen_loss"].append(g_loss_value)
        print("epoch:" + str(epoch))
        # グラフの描画（余裕があったら）
        if epoch % display_epoch == 0:
            save_metrics(loss, epoch)

        if epoch % param_save_epoch == 0:
            saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")

saving images
epoch:0
epoch:1
epoch:2
epoch:3
epoch:4
epoch:5
epoch:6
epoch:7
epoch:8
epoch:9
epoch:10
epoch:11
epoch:12
epoch:13
epoch:14
epoch:15
epoch:16
epoch:17
epoch:18
epoch:19
epoch:20
epoch:21
epoch:22
epoch:23
epoch:24
epoch:25
epoch:26
epoch:27
epoch:28
epoch:29
epoch:30
epoch:31
epoch:32
epoch:33
epoch:34
epoch:35
epoch:36
epoch:37
epoch:38
epoch:39
epoch:40
epoch:41
epoch:42
epoch:43
epoch:44
epoch:45
epoch:46
epoch:47
epoch:48
epoch:49
epoch:50
epoch:51
epoch:52
epoch:53
epoch:54
epoch:55
epoch:56
epoch:57
epoch:58
epoch:59
epoch:60
epoch:61
epoch:62
epoch:63
epoch:64
epoch:65
epoch:66
epoch:67
epoch:68
epoch:69
epoch:70
epoch:71
epoch:72
epoch:73
epoch:74
epoch:75
epoch:76
epoch:77
epoch:78
epoch:79
epoch:80
epoch:81
epoch:82
epoch:83
epoch:84
epoch:85
epoch:86
epoch:87
epoch:88
epoch:89
epoch:90
epoch:91
epoch:92
epoch:93
epoch:94
epoch:95
epoch:96
epoch:97
epoch:98
epoch:99
epoch:100
epoch:101
epoch:102
epoch:103
epoch:104
epoch:105
epoch:106
epoch:107
epoch:108
epoch:

KeyboardInterrupt: 

In [ ]:
def generator(batch_size, x):
    with tf.variable_scope("G"):
        x = tf.cast(x, tf.float32)
        x = tf.reshape(x, [batch_size, 100])

        with tf.variable_scope("fc1"):
            weights = tf.get_variable("fcw1", [100, 1024], initializer=tf.random_normal_initializer())
            biases = tf.get_variable("fcb1", [1024], initializer=tf.random_normal_initializer())
            fc1 = tf.add(tf.matmul(x, weights), biases)
            fc1 = tf.contrib.layers.batch_norm(fc1, center=True, scale=True, scope="bn")
            fc1 = tf.nn.relu(fc1)

        with tf.variable_scope("fc2"):
            weights = tf.get_variable("fcw1", [1024, 64*24*24],initializer=tf.random_normal_initializer())
            biases = tf.get_variable("fcb1", [64*24*24], initializer=tf.random_normal_initializer())
            fc2 = tf.matmul(fc1, weights)
            fc2 = tf.add(fc2, biases)
            fc2 = tf.contrib.layers.batch_norm(fc2, center=True, scale=True, scope="bn")
            fc2 = tf.nn.relu(fc2)
            fc2 = tf.reshape(fc2, [batch_size,24,24,64])

        with tf.variable_scope("deconv1"):
            weights = tf.get_variable("dcw1", [5, 5, 64, 64], initializer=tf.random_normal_initializer())
            biases = tf.get_variable("dcb1", [64], initializer=tf.random_normal_initializer())
            deconv1 = tf.nn.conv2d_transpose(fc2, weights, output_shape=[batch_size, 48, 48, 64], strides=[1, 2, 2, 1])
            deconv1 = tf.nn.bias_add(deconv1, biases)
            deconv1 = tf.contrib.layers.batch_norm(deconv1, center=True, scale=True, scope="bn")
            deconv1 = tf.nn.relu(deconv1)

        with tf.variable_scope("deconv2"):
            weights = tf.get_variable("dcw2", [5, 5, 3, 64], initializer=tf.random_normal_initializer())
            biases = tf.get_variable("dcb2", [3], initializer=tf.random_normal_initializer())
            deconv2 = tf.nn.conv2d_transpose(deconv1, weights, output_shape=[batch_size, 96, 96, 3], strides=[1,2,2,1])
            deconv2 = tf.nn.bias_add(deconv2, biases)
            deconv2 = tf.tanh(deconv2)
    return deconv2

In [ ]:
def lrelu(x, leak=0.2, name="lrelu"):
    with tf.variable_scope(name):
        f1 = 0.5 * (1 + leak)
        f2 = 0.5 * (1 - leak)
        return f1 * x + f2 * abs(x)

In [ ]:
def discriminator(x, dropout=0.5, reuse=False):

    x = tf.reshape(x, [-1, 96, 96, 3])
    with tf.variable_scope("D", reuse=reuse):
        with tf.variable_scope("conv1"):
            weights = tf.get_variable("cw1", [5, 5, 3, 64], initializer=tf.random_normal_initializer())
            biases = tf.get_variable("cb1", [64], initializer=tf.random_normal_initializer())
            conv1 = tf.nn.conv2d(x, weights, strides=[1, 2, 2, 1], padding='SAME')
            conv1 = tf.nn.bias_add(conv1, biases)
            conv1 = lrelu(conv1)

        with tf.variable_scope("conv2"):
            # Convolution
            weights = tf.get_variable("cw2", [5, 5, 64, 128], initializer=tf.random_normal_initializer())
            biases = tf.get_variable("cb2", [128], initializer=tf.random_normal_initializer())
            conv2 = tf.nn.conv2d(conv1, weights, strides=[1, 2, 2, 1], padding='SAME')
            conv2 = tf.nn.bias_add(conv2, biases)
            conv2 = lrelu(conv2)

        with tf.variable_scope("fc1"):
            weights = tf.get_variable("fcw1", [24*24*128, 256], initializer=tf.random_normal_initializer())
            biases = tf.get_variable("fcb1", [256], initializer=tf.random_normal_initializer())
            fc1 = tf.reshape(conv2, [-1, weights.get_shape().as_list()[0]])
            fc1 = tf.add(tf.matmul(fc1, weights), biases)
            fc1 = lrelu(fc1)
            fc1 = tf.nn.dropout(fc1, dropout)

        with tf.variable_scope("fc2"):
            weights = tf.get_variable("fcw2", [256,1], initializer=tf.random_normal_initializer())
            biases = tf.get_variable("fcb1", [1], initializer=tf.random_normal_initializer())
            fc2 = tf.add(tf.matmul(fc1, weights), biases)
            fc2 = tf.sigmoid(fc2)

    return fc2

In [ ]:
def discriminator_train(batch_size, output):
    # lossの定義
    clipvalue = tf.constant(1e-10, shape=[batch_size,1], dtype=tf.float32)
    loss = tf.log(tf.clip_by_value(output[0:batch_size], clipvalue, tf.ones(batch_size, tf.float32))) + tf.log(tf.clip_by_value(tf.ones(batch_size, tf.float32)-output[batch_size:2*batch_size], clipvalue, tf.ones(batch_size, tf.float32)))
    loss = tf.reduce_sum(loss, axis=1)
    loss = -tf.reduce_mean(loss)

    learning_rate = 1e-5
    beta1 = 0.1
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)


    grads_and_vars = optimizer.compute_gradients(loss)
    print()
    print("discriminator")
    for g,v in grads_and_vars:
        print(v.name)
    print()

    grads_and_vars = [[grad, var] for grad, var in grads_and_vars if var.name.startswith("D")]
    # grads_and_vars = [[grad, var] for grad, var in grads_and_vars if grad is not None]
    for g,v in grads_and_vars:
        print(v.name)
    train_op = optimizer.apply_gradients(grads_and_vars)

    return train_op, loss

In [ ]:
def generator_train(batch_size, output):
    # 全てが生成データ
    # discriの符号逆転バージョン
    #clipvalue = tf.constant(1e-10, shape=[batch_size,1], dtype=tf.float32)
    #loss = tf.log(tf.clip_by_value(tf.ones(batch_size, tf.float32)-output[0:batch_size], clipvalue, tf.ones(batch_size, tf.float32)))
    #loss = tf.reduce_sum(loss, axis=1)
    #loss = tf.reduce_mean(loss)    
    

    # lossの定義
    clipvalue = tf.constant(1e-10, shape=[batch_size,1], dtype=tf.float32)
    loss = -tf.reduce_sum(tf.log(tf.clip_by_value(output[0:batch_size], clipvalue, tf.ones(batch_size, tf.float32))), axis=1)
    loss = tf.reduce_mean(loss)
    
    learning_rate = 1e-5
    beta1 = 0.1
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1)
    
    grads_and_vars = optimizer.compute_gradients(loss)
    print()
    print("generator")
    for g,v in grads_and_vars:
        print(v.name)
    print()
    grads_and_vars = [[grad, var] for grad, var in grads_and_vars if var.name.startswith("G")]
    for g,v in grads_and_vars:
        print(v.name)
    #print(grads_and_vars)
    train_op = optimizer.apply_gradients(grads_and_vars)
    #train_op = optimizer.minimize(loss, grads_and_vars[1])

    return train_op, loss

In [ ]:
import numpy as np

X_train = np.load("irasutoya_10575x96x96x3_jpg.npy")
X_train = X_train/255

#X_train.shape

In [ ]:
epochs = 100
batch_size = 32
display_epoch = 50
save_param_epoch = 10000


# Network Parameters
n_input = 100 # MNIST data input (img shape: 28*28)
#n_classes = 10 # MNIST total classes (0-9 digits)
#dropout = 0.75 # Dropout, probability to keep units

# 結果
loss = {"dis_loss":[], "gen_loss":[]}
print(type(batch_size))

In [ ]:
#import tensorboard as tb
#tf.get_default_graph().get_operations()

#config = tf.ConfigProto(allow_soft_placement=True)
#config.gpu_options.allow_growth = True

p_noise = tf.placeholder(tf.float32, [None, n_input])
p_img_trainX = tf.placeholder(tf.float32, [None, 96, 96, 3])
p_batch_size = tf.placeholder(tf.int32)

generated_trainX = generator(p_batch_size, p_noise)

# 生成データと訓練データを結合
# X = np.vstack((img_trainX, generated_trainX))
X = tf.concat([p_img_trainX, generated_trainX], 0)

# ラベル作成
#y = np.ones(int(2*batch_size))
#y[batch_size:] = 0
#y = y.astype(int)

print()

dis = discriminator(X, reuse=False)
dis_train_op, dis_loss = discriminator_train(batch_size, dis)

print()

gan = discriminator(generated_trainX, reuse=True)
gen_train_op, gen_loss = generator_train(batch_size, gan)

# 画像可視化チェック用noise
noise_check = np.random.uniform(0, 1, size=[60, n_input])



with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
#    step = 1
    for epoch in range(epochs+1):
        print(epoch)
        
        # ノイズから生成データを生成
        noise = np.random.uniform(0, 1, size=[batch_size, n_input])

        if epoch % display_epoch == 0:
            gen_check_imgs = sess.run(generated_trainX, feed_dict={p_noise: noise_check, p_batch_size: 60})
            print("saving images")
            save_imgs(gen_check_imgs, epoch=epoch)

        # 訓練データを抜粋
        rand_train_index = np.random.randint(0, X_train.shape[0], size=batch_size)
        img_trainX = X_train[rand_train_index, :].astype(np.float32)
        #img_trainX = tf.constant(img_trainX)

        # discriminatorの学習
        _, d_loss = sess.run([dis_train_op, dis_loss], feed_dict={p_noise: noise, p_img_trainX: img_trainX, p_batch_size: batch_size})
        #sess.run(dis_train_op, feed_dict={p_noise: noise, p_img_trainX: img_trainX, p_batch_size: batch_size})

        # generatorの学習
        _, g_loss = sess.run([gen_train_op, gen_loss], feed_dict={p_noise: noise, p_batch_size: batch_size})
        #sess.run(gen_train_op, feed_dict={p_noise: noise, p_batch_size: batch_size})
        #gen_loss = sess.run(gen_loss, feed_dict={p_noise: noise})
        
        # 結果をappend
        loss["dis_loss"].append(d_loss)
        loss["gen_loss"].append(g_loss)

        # グラフの描画（余裕があったら）
        if epoch % display_epoch == 0:
            # Calculate batch loss and accuracy
            # d_loss, acc = sess.run([loss, accuracy], feed_dict)

            print("display_epoch:" + str(epoch))
            #print("dis_loss:" + str(dis_loss))
            #print("gen_loss:" + str(gen_loss))
            
            save_metrics(loss, epoch)
        # 訓練したgeneratorによる生成画像の可視化



In [ ]:
import tensorboard as tb
tb.show_graph(sess.graph)